<a href="https://colab.research.google.com/github/tolom131/Human-Activity-Recognition/blob/main/ensemble/tensorflow_supervised_ensemble_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from __future__ import print_function
from matplotlib import pyplot as plt
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, LSTM, Conv1D, Activation, MaxPooling1D, UpSampling1D, concatenate, Conv1DTranspose
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from tensorflow.keras import backend as K

import random as rn
from resource import *
import time
import math
import sys
from collections import Counter

In [2]:
from google.colab import drive 
drive.mount('/content/drive/')

directory_data  = './drive/MyDrive/HAR/'
filename_data   = 'WISDM_at_v2.0_raw.txt'

sys.path.append('/content/drive/MyDrive/HAR/')
import wisdm_2_0
# x_train, y_train, num_classes = wisdm_1_1.create_wisdm_1_1(directory_data + filename_data)
origianl_x, original_y, num_classes = wisdm_2_0.create_wisdm_2_0(directory_data + filename_data)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


x_train.shape :  (13913, 200, 3) y_train.shape:  (13913, 6)


In [3]:
from sklearn.utils import shuffle
def shift(x, y):
    new = []
    for i in range(5):
        temp = np.roll(x, i*40, axis=1)
        new.extend(temp)
    x = np.array(new)
    y = np.concatenate([y]*5)
    x, y = shuffle(x, y, random_state=42)
    return x,y

from scipy.interpolate import CubicSpline
def GenerateRandomCurves(X, sigma=0.2, knot=4):
    xx = (np.ones((X.shape[1], 1))*(np.arange(0, X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, X.shape[1]))
    x_range = np.arange(X.shape[0])

    cs_x = CubicSpline(xx[:, 0], yy[:, 0])
    cs_y = CubicSpline(xx[:, 1], yy[:, 1])
    cs_z = CubicSpline(xx[:, 2], yy[:, 2])
    return np.array([cs_x(x_range), cs_y(x_range), cs_z(x_range)]).transpose()

def DistortTimesteps(X, sigma=0.2):
    tt = GenerateRandomCurves(X, sigma)
    tt_cum = np.cumsum(tt, axis=0)

    t_scale = [(X.shape[0]-1)/tt_cum[-1, 0], (X.shape[0]-1)/tt_cum[-1, 1], (X.shape[0]-1)/tt_cum[-1, 2]]
    tt_cum[:, 0] = tt_cum[:, 0] * t_scale[0]
    tt_cum[:, 1] = tt_cum[:, 1] * t_scale[1]
    tt_cum[:, 2] = tt_cum[:, 2] * t_scale[2]

    return tt_cum

def DA_TimeWarp(X, sigma=0.2):
    tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(X.shape[0])

    X_new[:, 0] = np.interp(x_range, tt_new[:, 0], X[:, 0])
    X_new[:, 1] = np.interp(x_range, tt_new[:, 1], X[:, 1])
    X_new[:, 2] = np.interp(x_range, tt_new[:, 2], X[:, 2])

    return X_new

def TimeWarp(X, Y):
    for i in range(X.shape[0]):
        data = X[i, :, :]
        if i == 0:
            trans_list = DA_TimeWarp(data).reshape(-1, X.shape[1], X.shape[2])
        else:
            trans = DA_TimeWarp(data).reshape(-1, X.shape[1], X.shape[2])
            trans_list = np.concatenate([trans_list, trans], axis=0)

    return trans_list, Y

In [4]:
def build_resnet(input_shape, n_feature_maps, nb_classes):
    x = keras.layers.Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv1D(n_feature_maps, 8, 1, padding='same')(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    conv_y = keras.layers.Conv1D(n_feature_maps, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    conv_z = keras.layers.Conv1D(n_feature_maps, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps, 1, 1,padding='same')(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)

    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    x1 = y
    conv_x = keras.layers.Conv1D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    conv_y = keras.layers.Conv1D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    conv_z = keras.layers.Conv1D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)

    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    x1 = y
    conv_x = keras.layers.Conv1D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    conv_y = keras.layers.Conv1D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    conv_z = keras.layers.Conv1D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    full = keras.layers.GlobalAveragePooling1D()(y)
    out = keras.layers.Dense(nb_classes, activation='softmax', name="classified")(full)

    model = Model(inputs=x, outputs=out, name="classifier")
    return model

In [5]:
def CBR1d(filters, kernel_size=3, stride=1, name=None, bias=True):
    initializer = tf.keras.initializers.GlorotNormal()
    model = Sequential([
                        Conv1D(filters, kernel_size=kernel_size, strides=stride, padding="same", use_bias=True, kernel_initializer=initializer),
                        BatchNormalization(),
                        Activation("relu")
    ])
    return model

def Autoencoder():
    initializer = tf.keras.initializers.GlorotNormal()
    ###################################
    # encoder
    ###################################
    inputs  = Input(shape=(200, 3))
    enc1_1  = CBR1d(64)(inputs)
    enc1_2  = CBR1d(64)(enc1_1)
    pool1   = MaxPooling1D(2)(enc1_2)

    enc2_1  = CBR1d(128)(pool1)
    enc2_2  = CBR1d(128)(enc2_1)
    pool2   = MaxPooling1D(2)(enc2_2)
 
    enc3_1  = CBR1d(256)(pool2)
    enc3_2  = CBR1d(256)(enc3_1)
    pool3   = MaxPooling1D(2)(enc3_2)

    enc4_1  = CBR1d(512)(pool3)
    enc4_2  = CBR1d(512)(enc4_1)

    ###################################
    # decoder
    ###################################    
    dec5_1 = CBR1d(512)(enc4_2)
    cat4   = concatenate([enc4_2, dec5_1], axis=2)
    dec4_2 = CBR1d(512)(cat4)
    dec4_1 = CBR1d(256)(dec4_2)

    unpool3 = Conv1DTranspose(256, kernel_size=2, strides=2, kernel_initializer= initializer)(dec4_1)
    cat3    = concatenate([unpool3, enc3_2], axis=2)
    dec3_2  = CBR1d(256)(cat3)
    dec3_1 =  CBR1d(128)(dec3_2)

    unpool2 = Conv1DTranspose(128, kernel_size=2, strides=2, kernel_initializer= initializer)(dec3_1)
    cat3    = concatenate([unpool2, enc2_2], axis=2)
    dec2_2  = CBR1d(128)(cat3)
    dec2_1 =  CBR1d(64)(dec2_2)

    unpool1 = Conv1DTranspose(64, kernel_size=2, strides=2)(dec2_1)
    cat2    = concatenate([unpool1, enc1_2], axis=2)
    dec1_2  = CBR1d(128)(cat2)
    dec1_1 =  CBR1d(64)(dec1_2)
    decoded = Conv1D(3, kernel_size=1, strides=1, name="decoded", kernel_initializer= initializer)(dec1_1)

    encoder = Model(inputs=inputs, outputs=enc4_2, name="encoder")
    autoencoder = Model(inputs=inputs, outputs=decoded, name="decoder")
    
    return encoder, autoencoder

def Supervised_Autoencoder(x_train, y_train, x_test, y_test, epochs=150, alpha=0.1):
    earlystop = EarlyStopping(patience=200, monitor='val_loss', mode='min', restore_best_weights=True)
    filepath = "SAE.h5"
    checkpoint = ModelCheckpoint(filepath, verbose = 0, mode="min", save_best_only=True, save_weights_only = True)
    callbacks_list = [earlystop, checkpoint]
    adam = Adam(learning_rate=0.01)

    inputs = Input(shape=(200, 3))
    encoder, decoder = Autoencoder()
    classifier = build_resnet((25, 512), 64, 6)

    inputs  = Input(shape=(200, 3))
    encoded = encoder(inputs)
    decoded = decoder(inputs)
    classified = classifier(encoded)

    model = Model(inputs=[inputs], outputs=[classified, decoded])
    model.compile(optimizer=adam, loss=["categorical_crossentropy", "mse"], loss_weights=[1, alpha], metrics=["accuracy"])
    history = model.fit([x_train], [y_train, x_train], validation_data = ([x_val], [y_val, x_val]), batch_size = 64, epochs=epochs, verbose=1, callbacks=callbacks_list)
    return history, model, filepath    

In [6]:
TIME_PERIODS = 200
STEP = 200
N_FEATURES = 3


# 80%, 10%, 10%로 set 나누기
x_train, x_val, y_train, y_val = train_test_split(origianl_x, original_y, test_size=0.2, stratify=original_y)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, stratify=y_val)

# x_timewarp, y_timewarp  = TimeWarp(x_train, y_train)
# x_train,    y_train     = shift(x_train, y_train)
# x_train = np.concatenate([x_timewarp, x_train], axis=0)
# y_train = np.concatenate([y_timewarp, y_train], axis=0)
print("shape of x_train : ", x_train.shape)

shape of x_train :  (11130, 200, 3)


In [7]:
history, model, filepath = Supervised_Autoencoder(x_train, y_train, x_val, y_val, epochs=300, alpha=0.5)

Epoch 1/300
174/174 [==============================] - 26s 82ms/step - loss: 0.9008 - classifier_loss: 0.8767 - decoder_loss: 0.0482 - classifier_accuracy: 0.6868 - decoder_accuracy: 0.7587 - val_loss: 1.0371 - val_classifier_loss: 0.9951 - val_decoder_loss: 0.0841 - val_classifier_accuracy: 0.7009 - val_decoder_accuracy: 0.7039
Epoch 2/300
174/174 [==============================] - 11s 66ms/step - loss: 0.7208 - classifier_loss: 0.7170 - decoder_loss: 0.0076 - classifier_accuracy: 0.7462 - decoder_accuracy: 0.7976 - val_loss: 0.6794 - val_classifier_loss: 0.6700 - val_decoder_loss: 0.0188 - val_classifier_accuracy: 0.7168 - val_decoder_accuracy: 0.7544
Epoch 3/300
174/174 [==============================] - 11s 66ms/step - loss: 0.6542 - classifier_loss: 0.6515 - decoder_loss: 0.0054 - classifier_accuracy: 0.7603 - decoder_accuracy: 0.8057 - val_loss: 0.6383 - val_classifier_loss: 0.6129 - val_decoder_loss: 0.0507 - val_classifier_accuracy: 0.7735 - val_decoder_accuracy: 0.8560
Epoch 4

KeyboardInterrupt: ignored

In [ ]:
test_results = model.evaluate([x_test], [y_test, x_test])
print("maximum train acc : ", max(history.history["accuracy"]))
print("maximum valid acc : ", max(history.history["val_accuracy"]))
print("test acc : ", test_results[1])
print("test loss : ", test_results[0])


y_pred = model.predict([x_test])
score = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average="macro")
print("f1 score : ", score)
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)

label = ["Jogging", "LyingDown", "Sitting", "Stairs", "Stading", "Walking"]
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(matrix, interpolation="nearest")
fig.colorbar(cax)
ax.set_xticklabels(['']+label)
ax.set_yticklabels(['']+label)
plt.show()